In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# the goal of the notebook - to estimate lipschitz constant

from soul_gan.models.utils import load_gan
from soul_gan.utils.general_utils import DotConfig
from soul_gan.utils.general_utils import ROOT_DIR

import yaml
from pathlib import Path

import torch
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
import sys
sys.path.append("../thirdparty/studiogan/studiogan")
import studiogan
from soul_gan.models.studiogans import StudioDis, StudioGen

In [4]:
gan_config = yaml.safe_load(Path('../configs/gan_configs/cifar-10-wgan-gp-studio.yml').open('r'))
gan_config = DotConfig(gan_config['gan_config'])

In [5]:
device = torch.device(0 if torch.cuda.is_available() else 'cpu')

In [6]:
gen, dis = load_gan(gan_config, device=device)

  0%|          | 0/500 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 100.00 MiB (GPU 0; 10.92 GiB total capacity; 425.29 MiB already allocated; 20.44 MiB free; 526.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
def estimate_lipschitz_const(gen, dis, n_steps, batch_size):
    lipschitz_const_est = 0
    for step_id in range(n_steps):
        z = gen.prior.sample((batch_size,)).requires_grad_(True)
        x_fake = gen(z)
        dis_fake = dis(x_fake)
        energy = gen.prior.log_prob(z) + dis_fake
        grad = torch.autograd.grad(energy.sum(), z)[0]
        grad_norm = torch.norm(grad, dim=1).mean()
        lipschitz_const_est += grad_norm.item() / n_steps
    
    return lipschitz_const_est

In [10]:
lipschitz_const_est = estimate_lipschitz_const(gen, dis, 10, 100)

In [11]:
1./ lipschitz_const_est


0.023126984155831466